In [9]:
import pandas as pd 
import numpy as np
import json
import random
import pickle  
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package omw-1.4 to /home/henda/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/henda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/henda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Read dataset as json file

In [10]:
import os
for dirname, _, filenames in os.walk('/english'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [33]:
import yaml

dir_path = 'english/'
files_list = os.listdir(dir_path + os.sep)
files_list

['ai.yml',
 'botprofile.yml',
 'computers.yml',
 'conversations.yml',
 'emotion.yml',
 'food.yml',
 'gossip.yml',
 'greetings.yml',
 'health.yml',
 'history.yml',
 'humor.yml',
 'literature.yml',
 'money.yml',
 'movies.yml',
 'politics.yml',
 'psychology.yml',
 'science.yml',
 'sports.yml',
 'trivia.yml']

In [45]:
questions, answers, tags = [], [], []

for filepath in files_list:
    file_name = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file_name)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])
        tags.append(filepath)

In [48]:
english = pd.DataFrame({"tages":tags, "questions":questions, "answes":answers})
english

,tages,questions,answes
0,ai.yml,What is AI?,Artificial Intelligence is the branch of engin...
1,ai.yml,What is AI?,AI is the field of science which concerns itse...
2,ai.yml,Are you sentient?,Sort of.
3,ai.yml,Are you sentient?,By the strictest dictionary definition of the ...
4,ai.yml,Are you sentient?,Even though I'm a construct I do have a subjec...
...,...,...,...
581,trivia.yml,"The Hubble Space Telescope, launched into low ...",Edwin Hubble
582,trivia.yml,What is the name of the nearest major galaxy t...,The Andromeda Galaxy.
583,trivia.yml,God Save the Queen is the national anthem of w...,The United Kingdom of Great Britain
584,trivia.yml,"The Celtic Shelf, the seabed under the Celtic ...",Europe


In [55]:
english["tages"].str.split(".",expand=True)
english["tages"]

0          ai.yml
1          ai.yml
2          ai.yml
3          ai.yml
4          ai.yml
          ...    
581    trivia.yml
582    trivia.yml
583    trivia.yml
584    trivia.yml
585    trivia.yml
Name: tages, Length: 586, dtype: object

In [57]:
english['tages'] = english['tages'].str.split('.').str[0]
english

,tages,questions,answes
0,ai,What is AI?,Artificial Intelligence is the branch of engin...
1,ai,What is AI?,AI is the field of science which concerns itse...
2,ai,Are you sentient?,Sort of.
3,ai,Are you sentient?,By the strictest dictionary definition of the ...
4,ai,Are you sentient?,Even though I'm a construct I do have a subjec...
...,...,...,...
581,trivia,"The Hubble Space Telescope, launched into low ...",Edwin Hubble
582,trivia,What is the name of the nearest major galaxy t...,The Andromeda Galaxy.
583,trivia,God Save the Queen is the national anthem of w...,The United Kingdom of Great Britain
584,trivia,"The Celtic Shelf, the seabed under the Celtic ...",Europe


In [58]:
english.to_json(r'english.json')

In [24]:
with open('datas/data.json') as f:
    data = json.load(f)

In [31]:
def processing_json_dataset(dataset):
  tags = []
  inputs = []
  responses={}
  for intent in dataset['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
      inputs.append(lines)
      tags.append(intent['tag'])
  return [tags, inputs, responses]

In [32]:
[tags, inputs, responses] = processing_json_dataset(data)

# dataset as DataFrame

In [33]:
df = pd.DataFrame({"inputs":inputs, "tags":tags})
df

,inputs,tags
0,Hello,greetings
1,Hey,greetings
2,Hi,greetings
3,Good Day,greetings
4,Greetings,greetings
...,...,...
546,restlessness,Kidney stones
547,"Thank You, Goodbye",Goodbye
548,See you,Goodbye
549,Bye,Goodbye


# Pre-processing text of dataset

In [34]:
import string
df['inputs'] = df['inputs'].apply(lambda sequence: [ltrs.lower() for ltrs in sequence if ltrs not in string.punctuation])
df

,inputs,tags
0,"[h, e, l, l, o]",greetings
1,"[h, e, y]",greetings
2,"[h, i]",greetings
3,"[g, o, o, d, , d, a, y]",greetings
4,"[g, r, e, e, t, i, n, g, s]",greetings
...,...,...
546,"[r, e, s, t, l, e, s, s, n, e, s, s]",Kidney stones
547,"[t, h, a, n, k, , y, o, u, , g, o, o, d, b, ...",Goodbye
548,"[s, e, e, , y, o, u]",Goodbye
549,"[b, y, e]",Goodbye


In [36]:
df['inputs'] = df['inputs'].apply(lambda wrd: ''.join(wrd))
df

,inputs,tags
0,hello,greetings
1,hey,greetings
2,hi,greetings
3,good day,greetings
4,greetings,greetings
...,...,...
546,restlessness,Kidney stones
547,thank you goodbye,Goodbye
548,see you,Goodbye
549,bye,Goodbye


In [48]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['inputs'])
train = tokenizer.texts_to_sequences(df['inputs'])
features = pad_sequences(train)
le = LabelEncoder()
labels = le.fit_transform(df['tags'])

In [49]:
len(features[0])

9

In [50]:
input_shape = features.shape[1]
print(input_shape)

9


In [51]:
features.shape

(551, 9)

In [52]:
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
output_length = le.classes_.shape[0]
print("output length: ",output_length)

number of unique words :  516
output length:  97


In [53]:
tokenizer.word_index

{'in': 1,
 'pain': 2,
 'of': 3,
 'feeling': 4,
 'loss': 5,
 'to': 6,
 'the': 7,
 'swelling': 8,
 'skin': 9,
 'fever': 10,
 'breath': 11,
 'or': 12,
 'nose': 13,
 'appetite': 14,
 'difficulty': 15,
 'persistent': 16,
 'tiredness': 17,
 'stomach': 18,
 'eyes': 19,
 'coughing': 20,
 'swollen': 21,
 'chest': 22,
 'and': 23,
 'face': 24,
 'itchy': 25,
 'with': 26,
 'shortness': 27,
 'cough': 28,
 'joint': 29,
 'mouth': 30,
 'throat': 31,
 'dry': 32,
 'blocked': 33,
 'abdominal': 34,
 'weight': 35,
 'heart': 36,
 'back': 37,
 'up': 38,
 'blood': 39,
 'pale': 40,
 'on': 41,
 'problem': 42,
 'sore': 43,
 'neck': 44,
 'energy': 45,
 'hair': 46,
 'heartbeat': 47,
 'yellow': 48,
 'urine': 49,
 'severe': 50,
 'bad': 51,
 'vision': 52,
 'vomiting': 53,
 'cold': 54,
 'eye': 55,
 'red': 56,
 'urinate': 57,
 'muscle': 58,
 'diarrhoea': 59,
 'nausea': 60,
 'around': 61,
 'bleeding': 62,
 'pimples': 63,
 'lack': 64,
 'lower': 65,
 'night': 66,
 'is': 67,
 'legs': 68,
 'sweating': 69,
 'movement': 70,
 '

In [14]:
words = []
classes = []
documents = []
  
ignore_letters = ["?", "!", ".", ","]
  
for intent in data["entent"]:
    for pattern in intent["patterns"]:
        #tokenize each word
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        #add documents in the corpus
        documents.append((word_list, intent["tag"]))
        # add to our classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
# lemmaztize and lower each word and remove duplicates            
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
  
words = sorted(set(words))
classes = sorted(set(classes))

In [15]:
(len(documents), "documents")

(551, 'documents')

In [16]:
(len(classes), "classes", classes)

(97,
 'classes',
 ['Abdominal Migraine',
  'Abscessed Tooth',
  'Absence Seizure',
  'Achalasia',
  'Acne',
  'Acute Respiratory Distress Syndrome (ARDS)',
  'Allergies',
  'Alzheimers',
  'Amenorrhea',
  'Amyloidosis',
  'Anemia',
  'Ankylosing Spondylitis',
  'Anorexia',
  'Antitrypsin Deficiency',
  'Anxiety or Panic Disorder',
  'Aparaxia of Speech',
  'Appendicitis',
  'Arachnoiditis',
  'Arrhythmia',
  'Arthritis',
  'Asthma',
  'Attention Deficit Disorder (ADHD)',
  'Autism',
  'Bacterial Arthritis',
  'Bacterial Meningitis',
  'Benign Prostate Enlargement(BPE)',
  'Bipolar Disorder',
  'Blepharitis',
  'Bronchiectasis',
  'Bronchitis',
  'Bulimia',
  'Catarrh',
  'Cellulitis',
  'Chest Infection',
  'Chest Pain',
  'Chronic Fatigue Syndrome',
  'Common Cold',
  'Conjunctivitis',
  'Dehydration',
  'Dental Abscess',
  'Depression',
  'Diabetes',
  'Diarrhoea',
  'Dystonia',
  'Epilepsy',
  'Escherichia Coli',
  'Fetal alcohol spectrum disorder',
  'Flu',
  'Food poisoning',
  'F

In [17]:
(len(words), "unique lemmatized words", words)

(497,
 'unique lemmatized words',
 ['a',
  'abdomen',
  'abdominal',
  'abnormal',
  'abnormally',
  'about',
  'ache',
  'aching',
  'acid',
  'affected',
  'after',
  'all',
  'amount',
  'anal',
  'and',
  'anus',
  'anxiety',
  'anxious',
  'appetite',
  'area',
  'arm',
  'armpit',
  'around',
  'asking',
  'at',
  'attack',
  'attention',
  'awake',
  'back',
  'bad',
  'be',
  'beat',
  'before',
  'being',
  'belching',
  'belly',
  'blackhead',
  'blackout',
  'bladder',
  'bleeding',
  'blister',
  'bloatedness',
  'bloating',
  'block',
  'blocked',
  'blood',
  'blood-stained',
  'blue',
  'blurred',
  'blurry',
  'body',
  'bottom',
  'breath',
  'breathing',
  'breathlessness',
  'brittle',
  'brushing',
  'bulk',
  'burning',
  'bye',
  'ca',
  'change',
  'chest',
  'chesty',
  'chill',
  'choking',
  'clear',
  'clubbed',
  'coating',
  'cognition',
  'cold',
  'coming',
  'complex',
  'complexion',
  'concentrating',
  'concentration',
  'confused',
  'confusion',
  '

In [18]:
df = pd.DataFrame(documents)
df

,0,1
0,[Hello],greetings
1,[Hey],greetings
2,[Hi],greetings
3,"[Good, Day]",greetings
4,[Greetings],greetings
...,...,...
546,[restlessness],Kidney stones
547,"[Thank, You, ,, Goodbye]",Goodbye
548,"[See, you]",Goodbye
549,[Bye],Goodbye


In [19]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [20]:
dataset = []
template = [0]*len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(template)
    output_row[classes.index(document[1])] = 1
    dataset.append([bag, output_row])

random.shuffle(dataset)
dataset = np.array(dataset)

train_x = list(dataset[:, 0])
train_y = list(dataset[:, 1])



/home/henda/anaconda3/envs/final_project/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [21]:
df1 = pd.DataFrame(dataset)
df1

,0,1
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
546,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
547,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
548,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
549,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [22]:
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model.save("chatbot_model.h5", hist)

Epoch 1/200
111/111 [==============================] - 1s 4ms/step - loss: 4.5844 - accuracy: 0.0091
Epoch 2/200
111/111 [==============================] - 0s 2ms/step - loss: 4.5486 - accuracy: 0.0091
Epoch 3/200
111/111 [==============================] - 0s 2ms/step - loss: 4.5189 - accuracy: 0.0363
Epoch 4/200
111/111 [==============================] - 0s 2ms/step - loss: 4.4887 - accuracy: 0.0327
Epoch 5/200
111/111 [==============================] - 0s 2ms/step - loss: 4.4435 - accuracy: 0.0363
Epoch 6/200
111/111 [==============================] - 0s 2ms/step - loss: 4.3974 - accuracy: 0.0381
Epoch 7/200
111/111 [==============================] - 0s 2ms/step - loss: 4.3387 - accuracy: 0.0617
Epoch 8/200
111/111 [==============================] - 0s 2ms/step - loss: 4.2466 - accuracy: 0.0726
Epoch 9/200
111/111 [==============================] - 0s 2ms/step - loss: 4.1672 - accuracy: 0.0762
Epoch 10/200
111/111 [==============================] - 0s 3ms/step - loss: 4.0613 - accura

111/111 [==============================] - 0s 2ms/step - loss: 0.6735 - accuracy: 0.7387
Epoch 161/200
111/111 [==============================] - 0s 2ms/step - loss: 0.6493 - accuracy: 0.7568
Epoch 162/200
111/111 [==============================] - 0s 2ms/step - loss: 0.7207 - accuracy: 0.7387
Epoch 163/200
111/111 [==============================] - 0s 2ms/step - loss: 0.6654 - accuracy: 0.7713
Epoch 164/200
111/111 [==============================] - 0s 2ms/step - loss: 0.7073 - accuracy: 0.7441
Epoch 165/200
111/111 [==============================] - 0s 2ms/step - loss: 0.7057 - accuracy: 0.7495
Epoch 166/200
111/111 [==============================] - 0s 2ms/step - loss: 0.6699 - accuracy: 0.7405
Epoch 167/200
111/111 [==============================] - 0s 2ms/step - loss: 0.6689 - accuracy: 0.7586
Epoch 168/200
111/111 [==============================] - 0s 2ms/step - loss: 0.6828 - accuracy: 0.7477
Epoch 169/200
111/111 [==============================] - 0s 2ms/step - loss: 0.6782 - a

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               127488    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 97)                12513     
Total params: 172,897
Trainable params: 172,897
Non-trainable params: 0
_________________________________________________________________


In [24]:
score = model.evaluate(np.array(train_x), np.array(train_y), verbose=0)
score

[0.4230034351348877, 0.7949183583259583]